In [1]:
using Distributed
addprocs(10)

10-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

In [2]:
nprocs()

11

In [3]:
@everywhere using DrWatson
@everywhere @quickactivate :Catalyst
@everywhere using BenchmarkTools

@everywhere function benchmark_distributed_evaluation(parameters,N)
    errors = zeros(N) # init model errors
    # we need to read the experiment data in this block
    input_exp = Float64[]
    output_exp = Float64[]
    for row in CSV.File(datadir("experiment/new-data.csv"); delim = " ")
         push!(input_exp, row.I)
         push!(output_exp, row.O)
    end
    # done reading 
    
    F = Array{Future}(undef,N)
    
    for i in 1:N 
        F[i] = @spawnat i+1 (Catalyst.solve(parameters[1], parameters[2], 1.,
                                      input_exp, output_exp, progress=false, 
                                      microcomp_type=:nonlinear,
                                      Q=parameters[3], kₙ=parameters[4],
                                      calibration=true))
    end 
    
    for i in 1:N
        errors[i] = fetch(F[i])
    end
    return errors
end

parameters = [1e-4, 1., 1., 10.]
@time benchmark_distributed_evaluation(parameters,10)

┌ Info: Precompiling Catalyst [01e5440a-74e3-4b45-b8b9-c13ddfc5051b]
└ @ Base loading.jl:1273


      From worker 6:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 11:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 3:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 2:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 9:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 5:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 10:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 7:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 4:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 8:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml

┌ Warning: Module PlotThemes with build ID 456389398202242 is missing from the cache.
│ This may mean PlotThemes [ccf2f8ad-2431-5c83-bf29-c5338b663b6a] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1011
┌ Warning: Module PlotThemes with build ID 456389398202242 is missing from the cache.
│ This may mean PlotThemes [ccf2f8ad-2431-5c83-bf29-c5338b663b6a] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1011
┌ Info: Skipping precompilation since __precompile__(false). Importing Catalyst [01e5440a-74e3-4b45-b8b9-c13ddfc5051b].
└ @ Base loading.jl:1028
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1273
┌ Warning: Module PlotThemes with build ID 456389398202242 is missing from the cache.
│ This may mean PlotThemes [ccf2f8ad-2431-5c83-bf29-c5338b663b6a] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1011
┌ Info: Skipping pre

      From worker 6:	┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
      From worker 6:	│   exception = Required dependency PlotThemes [ccf2f8ad-2431-5c83-bf29-c5338b663b6a] failed to load from a cache file.
      From worker 6:	└ @ Base loading.jl:1036
      From worker 5:	┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
      From worker 5:	│   exception = Required dependency PlotThemes [ccf2f8ad-2431-5c83-bf29-c5338b663b6a] failed to load from a cache file.
      From worker 5:	└ @ Base loading.jl:1036
      From worker 7:	┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
      From worker 7:	│   exception = Required dependency PlotThemes [ccf2f8ad-2431-5c83-bf29-c5338b663b6a] failed to load from a cache file.
      From worker 7:

10-element Array{Float64,1}:
 183.03812687208995
 183.03812687208995
 183.03812687208995
 183.03812687208995
 183.03812687208995
 183.03812687208995
 183.03812687208995
 183.03812687208995
 183.03812687208995
 183.03812687208995